In [1]:
#get all excel files in the data directory
import os
excel_files = [f for f in os.listdir('cleaned-data') if f.endswith('.xlsx')]
print(excel_files)

['cleaned-lp1.xls( failure1).xlsx', 'cleaned-lp3.xls(position of part after a transfer failure).xlsx', 'cleaned-lp4.txt(failures in approach to ungrasp position).xlsx', 'cleaned-lp5.txt( failures in motion with part).xlsx', 'lp2_cleaned.xlsx']


In [2]:
import pandas as pd
#LP1: Failures in Approach to Grasp Position
# Method 1: Using a raw string
file_path = f'cleaned-data/{excel_files[2]}'


# Load the Excel file
df = pd.read_excel(file_path)
print(df.head())

    class  Fx  Fy  Fz  Tx  Ty  Tz  class-oridnal
0  normal  -2   2  20   5  -6  -1              1
1  normal  -2   1  20   5  -6   0              1
2  normal  -2   1  23   5  -6   0              1
3  normal  -2   2  20   5  -6  -1              1
4  normal  -3   1  22   5  -6  -1              1


In [3]:
# Drop rows with NaN values in the relevant columns
cleaned_df = df.dropna(subset=['Fx', 'Fy', 'Tx', 'Ty'])

In [4]:
# Convert columns to float
cleaned_df[['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz']] = cleaned_df[['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz']].astype(float)

In [5]:
#normalizing the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cleaned_df[['Fx', 'Fy', 'Tx', 'Ty']] = scaler.fit_transform(cleaned_df[['Fx', 'Fy', 'Tx', 'Ty']])
print(cleaned_df.head())

    class        Fx        Fy    Fz        Tx        Ty   Tz  class-oridnal
0  normal  0.655019  0.468293  20.0  0.450543  0.598668 -1.0              1
1  normal  0.655019  0.467073  20.0  0.450543  0.598668  0.0              1
2  normal  0.655019  0.467073  23.0  0.450543  0.598668  0.0              1
3  normal  0.655019  0.468293  20.0  0.450543  0.598668 -1.0              1
4  normal  0.654275  0.467073  22.0  0.450543  0.598668 -1.0              1


In [6]:
import plotly.graph_objects as go

# 3D scatter plot for Force (Fx, Fy, Fz)
force_plot = go.Scatter3d(
    x=cleaned_df['Fx'],
    y=cleaned_df['Fy'],
    z=cleaned_df['Fz'],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.8),
    name='Force'
)

# 3D scatter plot for Torque (Tx, Ty, Tz)
torque_plot = go.Scatter3d(
    x=cleaned_df['Tx'],
    y=cleaned_df['Ty'],
    z=cleaned_df['Tz'],
    mode='markers',
    marker=dict(size=5, color='orange', opacity=0.8),
    name='Torque'
)

# Create subplots
fig = go.Figure()

# Add Force plot
fig.add_trace(force_plot)
fig.add_trace(torque_plot)

# Layout settings
fig.update_layout(
    title='3D Scatter Plots for Force and Torque',
    scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis'
    ),
    legend=dict(x=0.8, y=1.2),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()


In [7]:
#split the data based on the class column into seperated dataframes
# find the unique values in the class column
unique_classes = cleaned_df['class'].unique()
unique_classes

array(['normal', 'collision', 'obstruction'], dtype=object)

In [8]:
# Create a dictionary to store the dataframes
class_dfs = {}
# Split the data based on the class column
for class_name in unique_classes:
    class_dfs[class_name] = cleaned_df[cleaned_df['class'] == class_name]

# Display the first few rows of each class
for class_name, class_df in class_dfs.items():
    print(f'Class: {class_name}')
    print(class_df.head())
    print('\n')

Class: normal
    class        Fx        Fy    Fz        Tx        Ty   Tz  class-oridnal
0  normal  0.655019  0.468293  20.0  0.450543  0.598668 -1.0              1
1  normal  0.655019  0.467073  20.0  0.450543  0.598668  0.0              1
2  normal  0.655019  0.467073  23.0  0.450543  0.598668  0.0              1
3  normal  0.655019  0.468293  20.0  0.450543  0.598668 -1.0              1
4  normal  0.654275  0.467073  22.0  0.450543  0.598668 -1.0              1


Class: collision
         class        Fx        Fy     Fz        Tx        Ty    Tz  \
360  collision  0.645353  0.492683  123.0  0.394565  0.604496 -15.0   
361  collision  0.678067  0.489024   13.0  0.460326  0.652789  17.0   
362  collision  0.671375  0.448780   38.0  0.448370  0.640300  -6.0   
363  collision  0.662454  0.451220   23.0  0.456522  0.609492  -2.0   
364  collision  0.655019  0.467073   16.0  0.449457  0.609492  -2.0   

     class-oridnal  
360              0  
361              0  
362              0  


In [9]:

import plotly.graph_objects as go
class_name = unique_classes[0]
df_fr_collision=class_dfs[class_name]
# 3D scatter plot for Force (Fx, Fy, Fz)
force_plot = go.Scatter3d(
    x=df_fr_collision['Fx'],
    y=df_fr_collision['Fy'],
    z=df_fr_collision['Fz'],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.8),
    name='Force'
)

# 3D scatter plot for Torque (Tx, Ty, Tz)
torque_plot = go.Scatter3d(
    x=df_fr_collision['Tx'],
    y=df_fr_collision['Ty'],
    z=df_fr_collision['Tz'],
    mode='markers',
    marker=dict(size=5, color='orange', opacity=0.8),
    name='Torque'
)

# Create subplots
fig = go.Figure()

# Add Force plot
fig.add_trace(force_plot)
fig.add_trace(torque_plot)

# Layout settings
fig.update_layout(
    title='3D Scatter Plots for Force and Torque for class: ' + class_name,
    title_font=dict(size=14),  # Change the title font size
    scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis'
    ),
    legend=dict(x=0.8, y=1.2),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()


In [10]:

import plotly.graph_objects as go
class_name = unique_classes[1]
df_fr_collision=class_dfs[class_name]
# 3D scatter plot for Force (Fx, Fy, Fz)
force_plot = go.Scatter3d(
    x=df_fr_collision['Fx'],
    y=df_fr_collision['Fy'],
    z=df_fr_collision['Fz'],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.8),
    name='Force'
)

# 3D scatter plot for Torque (Tx, Ty, Tz)
torque_plot = go.Scatter3d(
    x=df_fr_collision['Tx'],
    y=df_fr_collision['Ty'],
    z=df_fr_collision['Tz'],
    mode='markers',
    marker=dict(size=5, color='orange', opacity=0.8),
    name='Torque'
)

# Create subplots
fig = go.Figure()

# Add Force plot
fig.add_trace(force_plot)
fig.add_trace(torque_plot)

# Layout settings
fig.update_layout(
    title='3D Scatter Plots for Force and Torque for class: ' + class_name,
    title_font=dict(size=14),  # Change the title font size
    scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis'
    ),
    legend=dict(x=0.8, y=1.2),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()


In [11]:

import plotly.graph_objects as go
class_name = unique_classes[2]
df_fr_collision=class_dfs[class_name]
# 3D scatter plot for Force (Fx, Fy, Fz)
force_plot = go.Scatter3d(
    x=df_fr_collision['Fx'],
    y=df_fr_collision['Fy'],
    z=df_fr_collision['Fz'],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.8),
    name='Force'
)

# 3D scatter plot for Torque (Tx, Ty, Tz)
torque_plot = go.Scatter3d(
    x=df_fr_collision['Tx'],
    y=df_fr_collision['Ty'],
    z=df_fr_collision['Tz'],
    mode='markers',
    marker=dict(size=5, color='orange', opacity=0.8),
    name='Torque'
)

# Create subplots
fig = go.Figure()

# Add Force plot
fig.add_trace(force_plot)
fig.add_trace(torque_plot)

# Layout settings
fig.update_layout(
    title='3D Scatter Plots for Force and Torque for class: ' + class_name,
    title_font=dict(size=14),  # Change the title font size
    scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis'
    ),
    legend=dict(x=0.8, y=1.2),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()


In [13]:
from plotly.subplots import make_subplots

import plotly.graph_objects as go

# Create a subplot figure with 2 rows and 2 columns
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],
           [{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
    subplot_titles=unique_classes
)

# Mapping of class names to subplot positions
subplot_positions = {
    unique_classes[0]: (1, 1),
    unique_classes[1]: (1, 2),
    unique_classes[2]: (2, 1),
}

# Iterate through each class and add a 3D scatter plot to the corresponding subplot
for class_name, class_df in class_dfs.items():
    row, col = subplot_positions[class_name]
    force_plot = go.Scatter3d(
        x=class_df['Fx'],
        y=class_df['Fy'],
        z=class_df['Fz'],
        mode='markers',
        marker=dict(size=5, color='blue', opacity=0.8),
        name=f'{class_name} Force'
    )
    torque_plot = go.Scatter3d(
        x=class_df['Tx'],
        y=class_df['Ty'],
        z=class_df['Tz'],
        mode='markers',
        marker=dict(size=5, color='orange', opacity=0.8),
        name=f'{class_name} Torque'
    )
    fig.add_trace(force_plot, row=row, col=col)
    fig.add_trace(torque_plot, row=row, col=col)

# Update layout settings
fig.update_layout(
    title='3D Scatter Plots for Force and Torque by Class',
    height=800,
    showlegend=False
)

fig.show()
